In [1]:
import pandas as pd
import os
import re
import csv
import jieba

In [2]:
corpus = pd.read_csv('/Users/lola/Manning_DATA490/corpus_metadata.tsv', sep='\t') #reading in corpus into a data frame
corpus = corpus.iloc[:, :10]
corpus['id'] = corpus['id'].astype(str)
corpus

,id,title,genre,subcat,author,era,origin,originid,simptrad,notes
0,0,周易(正文),經,易類,NaN,周,kanseki,KR1a0001,trad,NaN
1,1,子夏易傳,經,易類,卜商,周,kanseki,KR1a0002,trad,NaN
2,2,周易鄭康成註,經,易類,鄭玄,汉,kanseki,KR1a0003,trad,NaN
3,3,增補鄭氏周易,經,易類,鄭玄,汉,kanseki,KR1a0004,trad,NaN
4,4,陸氏易解,經,易類,陸績,吴,kanseki,KR1a0005,trad,NaN
...,...,...,...,...,...,...,...,...,...,...
32474,46027,竹齋集,集,別集類,王冕,明,gutenberg,7220_ZHU ZHAI JI (BAMBOO HOUSE),trad,NaN
32475,46028,莊子的故事,NaN,NaN,宋曄,NaN,gutenberg,23913_ZHUANG-ZI'S STORY,trad,NaN
32476,46029,駐春園小史,集,小说,吳航野客,NaN,gutenberg,27328_ZHUCHUNYUAN XIAOSHI,trad,NaN
32477,46030,左傳,NaN,NaN,左丘明,春秋,gutenberg,24136_ZUO ZHUAN,trad,NaN


In [4]:
REMOVE = [
    "』",
    "。",
    "！",
    "，",
    "：",
    "、",
    "（",
    "）",
    "；",
    "？",
    "〉",
    "〈",
    "」",
    "「",
    "『",
    "“",
    "”",
    "!",
    '"',
    "#",
    "$",
    "%",
    "&",
    "'",
    "(",
    ")",
    "*",
    "+",
    ",",
    "-",
    ".",
    "/",
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    ":",
    ";",
    "<",
    "=",
    ">",
    "?",
    "@",
    "[",
    "\\",
    "]",
    "_",
    "`",
    "{",
    "|",
    "}",
    "~",
    "¤",
    "±",
    "·",
    "×",
    "à",
    "á",
    "è",
    "é",
    "ê",
    "ì",
    "í",
    "ò",
    "ó",
    "÷",
    "ù",
    "ú",
    "ü",
    "ā",
    "ī",
    "ń",
    "ň",
    "ō",
    "ū",
    "ǎ",
    "ǐ",
    "ǔ",
    "ǖ",
    "ǘ",
    "ǚ",
    "ǜ",
    "ǹ",
    "ɑ",
    "ɡ",
    "α",
    "β",
    "γ",
    "ε",
    "ζ",
    "η",
    "θ",
    "ι",
    "κ",
    "λ",
    "μ",
    "ν",
    "ξ",
    "ο",
    "π",
    "ρ",
    "σ",
    "τ",
    "υ",
    "φ",
    "χ",
    "ψ",
    "ω",
    "а",
    "б",
    "в",
    "г",
    "д",
    "е",
    "к",
    "о",
    "п",
    "р",
    "с",
    "т",
    "у",
    "ф",
    "х",
    "ц",
    "ч",
    "ш",
    "щ",
    "ъ",
    "ы",
    "ь",
    "э",
    "ю",
    "я",
    "ё",
    "—",
    "‖",
    "‘",
    "’",
    "…",
    "※",
    "ⅰ",
    "ⅲ",
    "∈",
    "∏",
    "∑",
    "√",
    "∠",
    "∥",
    "∧",
    "∩",
    "∪",
    "∫",
    "∮",
    "∶",
    "∷",
    "∽",
    "≈",
    "≌",
    "≡",
    "⊙",
    "⊥",
    "⌒",
    "①",
    "②",
    "③",
    "④",
    "⑤",
    "⑥",
    "⑦",
    "⑧",
    "⑨",
    "⑩",
    "⑴",
    "⑵",
    "⑶",
    "⑷",
    "⑸",
    "⑹",
    "⑺",
    "⑻",
    "⑼",
    "⑽",
    "⑾",
    "⑿",
    "⒀",
    "⒁",
    "⒂",
    "⒃",
    "⒄",
    "⒅",
    "⒆",
    "⒈",
    "⒉",
    "⒊",
    "⒋",
    "⒌",
    "⒍",
    "⒎",
    "⒏",
    "⒐",
    "⒑",
    "⒒",
    "⒓",
    "⒔",
    "⒕",
    "⒖",
    "⒗",
    "⒘",
    "⒙",
    "⒚",
    "⒛",
    "─",
    "┅",
    "┋",
    "┌",
    "┍",
    "┎",
    "┏",
    "┐",
    "┑",
    "┒",
    "┓",
    "└",
    "┕",
    "┘",
    "┙",
    "┚",
    "┛",
    "├",
    "┝",
    "┞",
    "┠",
    "┡",
    "┢",
    "┣",
    "┤",
    "┥",
    "┦",
    "┧",
    "┩",
    "┪",
    "┫",
    "┬",
    "┭",
    "┮",
    "┯",
    "┰",
    "┱",
    "┲",
    "┳",
    "■",
    "□",
    "▲",
    "△",
    "◆",
    "◇",
    "○",
    "◎",
    "●",
    "★",
    "︶",
    "﹑",
    "﹔",
    "﹖",
    "＂",
    "＃",
    "％",
    "＆",
    "＊",
    "．",
    "／",
    "０",
    "１",
    "２",
    "３",
    "４",
    "５",
    "６",
    "７",
    "８",
    "９",
    "＜",
    "＝",
    "＞",
    "＠",
    "［",
    "＼",
    "］",
    "＿",
    "｀",
    "ａ",
    "ｂ",
    "ｃ",
    "ｄ",
    "ｅ",
    "ｆ",
    "ｇ",
    "ｈ",
    "ｉ",
    "ｊ",
    "ｋ",
    "ｌ",
    "ｍ",
    "ｎ",
    "ｏ",
    "ｐ",
    "ｑ",
    "ｒ",
    "ｓ",
    "ｔ",
    "ｕ",
    "ｖ",
    "ｗ",
    "ｘ",
    "ｙ",
    "ｚ",
    "｛",
    "｝",
    "～",
    "￥",
    "a",
    "b",
    "c",
    "d",
    "e",
    "f",
    "g",
    "h",
    "i",
    "j",
    "k",
    "l",
    "m",
    "n",
    "o",
    "p",
    "q",
    "r",
    "s",
    "t",
    "u",
    "v",
    "w",
    "x",
    "y",
    "z",
    "《",
    "》",
    "〔",
    "〕",
    "【",
    "】",
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "Ｗ",
    "Ｘ",
    "Ｙ",
    "Ｚ",
    "＾",
    "｜",
    "￠",
    "￡",
    "~",
    "¶",
    "W",
    "Y",
    "Z",
    "^",
    "|",
    "丨"

]

def clean(content, remove=REMOVE, remove_whitespace=True, simplified=False):
    # These two lines are useful for Chinese texts where there was no whitespace or punctuation
    # in the original documents
    if remove_whitespace:
        content = re.sub("\s+", "", content)
        content = re.sub("\n+", "", content)
    content = content.translate(str.maketrans("", "", "".join(remove)))
    return content

In [12]:
input_folder = '/Users/lola/Manning_DATA490/texts'
output_folder = '/Users/lola/Manning_DATA490/cleaned_texts'

files_with_decoding_errors = []

for file in os.listdir(input_folder):
    # Skip files starting with a dot (hidden files and .DS_store
    if file.startswith('.'):
        continue

    input_filepath = os.path.join(input_folder, file)
    output_filepath = os.path.join(output_folder, file)  # move cleaned file to 'cleaned_texts' folder


    with open(input_filepath, 'r', encoding = 'utf-8', errors = 'replace') as file:
        content = file.read()

        chinese_chars_only = clean(content)
              
    with open(output_filepath, 'w', encoding='utf-8') as output_file:
        output_file.write(chinese_chars_only)

In [19]:
folder_path = '/Users/lola/Manning_DATA490/cleaned_texts'

# Get the list of files in the folder
files_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Initialize lists to store file names and lengths
file_names = []
file_lengths = []

# Iterate through each file
for file_name in files_list:
    file_path = os.path.join(folder_path, file_name)

    # Get the length of the file
    file_size = os.path.getsize(file_path)

    file_name_without_extension = os.path.splitext(file_name)[0]

    # Append modified file name and length to respective lists
    file_names.append(file_name_without_extension)
    file_lengths.append(file_size)

# Create a DataFrame from the lists
file_data = {'id': file_names, 'length': file_lengths}
file_df = pd.DataFrame(file_data)

32478


In [ ]:
merged_df = pd.merge(corpus, file_df, on='id', how='inner') #merge corpus df with the new file id

In [ ]:
selected_columns = ['id', 'title','genre','author','era','length'] #select columns wanted from df
selected_df = merged_df[selected_columns]

Add Tokenization Ratios to DataFrame

In [5]:
#n-gram tokenizer without spaces
def n_gram_tokenizer(text, n = 1):
    return [text[i:i+n] for i in range(len(text) - (n - 1))]

In [6]:
#get ratio
def get_ratio(result):
    return len(set(result))/len(result)

In [7]:
#add spaces
def add_spaces(result):
    return " ".join(result)

In [8]:
#main function
def ratio(text, n = 1):
    result = n_gram_tokenizer(text, n)
    return get_ratio(result)

In [2]:
#main function for jieba
def ratio_jieba(text):
    result = list(jieba.cut(text))  # Tokenize the text using jieba.cut
    return get_ratio(result)  # Calculate the ratio using the get_ratio function


In [ ]:
#add 1 gram to df
folder_path = '/Users/lola/Manning_DATA490/cleaned_texts'

ids = []
ratios = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    if file_name.startswith('.'):
        continue
    
    with open(file_path, 'r') as file:
        file_content = file.read()
    
    # Apply the function ratio() to the content of each file
    result = ratio(file_content)
    
    # Extract the id from the file_name (assuming the file_name is the id)
    file_id = file_name.split('.')[0]
    #try:
        #file_id = int(file_name.split('.')[0])  # Assuming the file name is in the format "id.txt"
    #except ValueError:
        #pass  # Skip files that do not contain numeric identifiers
    
    ids.append(file_id)
    ratios.append(result)

# Create a DataFrame from the dictionary
df1 = pd.DataFrame({'id': ids, '1 Gram Ratio': ratios})

In [ ]:
merged_1 = pd.merge(selected_df, df1, on='id', how='inner') #merge with main df

In [ ]:
#add 2 gram to df
folder_path = '/Users/lola/Manning_DATA490/cleaned_texts'

ids = []
ratios = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    with open(file_path, 'r') as file:
        file_content = file.read()
    
    # Apply the function ratio() to the content of each file
    result = ratio(file_content,2)
    
    # Extract the id from the file_name (assuming the file_name is the id)
    file_id = None
    try:
        file_id = int(file_name.split('.')[0])  # Assuming the file name is in the format "id.txt"
    except ValueError:
        pass  # Skip files that do not contain numeric identifiers
    
    ids.append(file_id)
    ratios.append(result)

# Create a DataFrame from the dictionary
df2 = pd.DataFrame({'id': ids, '2 Gram Ratio': ratios}) 

In [ ]:
merged_2 = pd.merge(merged_1, df2, on='id', how='inner') #merge with main df

In [9]:
#add jieba to df
folder_path = '/Users/lola/Manning_DATA490/cleaned_texts'

ids = []
ratios = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    with open(file_path, 'r') as file:
        file_content = file.read()
    
    # Apply the function ratio() to the content of each file
    result = ratio_jieba(file_content)
    
    # Extract the id from the file_name (assuming the file_name is the id)
    file_id = None
    try:
        file_id = int(file_name.split('.')[0])  # Assuming the file name is in the format "id.txt"
    except ValueError:
        pass  # Skip files that do not contain numeric identifiers
    
    ids.append(file_id)
    ratios.append(result)

# Create a DataFrame from the dictionary
dfjieba = pd.DataFrame({'id': ids, 'Jieba Ratio': ratios})

IsADirectoryError: [Errno 21] Is a directory: '/Users/lola/Manning_DATA490/cleaned_texts/.ipynb_checkpoints'

In [ ]:
full_df = pd.merge(merged_2, dfjieba, on='id', how='inner') #merge with main df

In [ ]:
print(full_df)

Download 'genres' and 'texts' 

In [ ]:
folder_path = '/Users/lola/Manning_DATA490/cleaned_texts' #path to folder of cleaned texts

genres = []
texts = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    if file_name.startswith('.'): #skip file if starts with '.'
        continue
    
    with open(file_path, 'r') as file:
        file_content = file.read()
        
    genre_index = file_name.split('.')[0]  # Extract genre index from file name
    
    if len(genre_index) > 0:
        genre_col = classify_df.loc[classify_df['id'] == str(genre_index)]
        if not genre_col.empty:
            genre = genre_col['genre'].iloc[0]  # Get the genre value from DataFrame
        else:
            print(f"Genre not found for index {genre_index}")
            continue
    else:
        print("Genre index not found")
        continue
     
    # Apply preprocessing to individual document
    file_content = pre_process(file_content)
    
    genres.append(genre)
    texts.append(file_content)